# lgbm-tfidf 도전

In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import warnings
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import os

In [10]:
warnings.filterwarnings(action='ignore')
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [16]:
path = './data'
trn = pd.read_csv(os.path.join(path,'train.csv'),encoding = 'utf-8')
tst = pd.read_csv(os.path.join(path,'test_x.csv'),encoding = 'utf-8')
sample_submission = pd.read_csv(os.path.join(path,'sample_submission.csv'),encoding = 'utf-8')

#trn = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\train.csv', encoding = 'utf-8')
#tst = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\test_x.csv', encoding = 'utf-8')
#sample_submission = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\sample_submission.csv', encoding = 'utf-8')

[문자열 가공]  
1. Bag-of-Words(simple):countVectorizer, TfidVectorizer  
2. Hashing Trick: hashingVectorizer  
3. Embeddings(with 신경망): Word2Vec, GloVe

## Bag-of-Words_Count 피처 생성1

In [27]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
X_tst = vec.fit_transform(tst['text'])
print(X_cnt.shape, X_tst.shape)

(54879, 2892) (19617, 1990)


In [28]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [30]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Lgbm 모델 학습_count

In [33]:
y=trn.author.values
X_cnt = X_cnt.astype('float32') 
y = y.astype('float32')
X_tst = X_tst.astype('float32')   
p = np.zeros((X_cnt.shape[0], 5))
p_tst = np.zeros((X_tst.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(X_cnt, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=3000,
                             num_leaves=256,
                             learning_rate=0.009,
                             subsample=.8,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.6,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(X_cnt[i_trn], y[i_trn],eval_set=[(X_cnt[i_val], y[i_val])], eval_metric='multiclass',early_stopping_rounds=15)
    p[i_val, :] = clf.predict_proba(X_cnt[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(X_tst) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

training model for CV #1
[1]	valid_0's multi_logloss: 1.55939
Training until validation scores don't improve for 15 rounds
[2]	valid_0's multi_logloss: 1.54824
[3]	valid_0's multi_logloss: 1.53794
[4]	valid_0's multi_logloss: 1.52787
[5]	valid_0's multi_logloss: 1.51826
[6]	valid_0's multi_logloss: 1.50833
[7]	valid_0's multi_logloss: 1.49876
[8]	valid_0's multi_logloss: 1.48943
[9]	valid_0's multi_logloss: 1.48034
[10]	valid_0's multi_logloss: 1.47112
[11]	valid_0's multi_logloss: 1.46225
[12]	valid_0's multi_logloss: 1.45356
[13]	valid_0's multi_logloss: 1.44575
[14]	valid_0's multi_logloss: 1.43761
[15]	valid_0's multi_logloss: 1.42952
[16]	valid_0's multi_logloss: 1.4214
[17]	valid_0's multi_logloss: 1.41366
[18]	valid_0's multi_logloss: 1.40599
[19]	valid_0's multi_logloss: 1.39846
[20]	valid_0's multi_logloss: 1.39101
[21]	valid_0's multi_logloss: 1.38326
[22]	valid_0's multi_logloss: 1.37611
[23]	valid_0's multi_logloss: 1.36894
[24]	valid_0's multi_logloss: 1.36176
[25]	valid_0

[210]	valid_0's multi_logloss: 0.849276
[211]	valid_0's multi_logloss: 0.848168
[212]	valid_0's multi_logloss: 0.847028
[213]	valid_0's multi_logloss: 0.845886
[214]	valid_0's multi_logloss: 0.844742
[215]	valid_0's multi_logloss: 0.84366
[216]	valid_0's multi_logloss: 0.842571
[217]	valid_0's multi_logloss: 0.841456
[218]	valid_0's multi_logloss: 0.840341
[219]	valid_0's multi_logloss: 0.839218
[220]	valid_0's multi_logloss: 0.838215
[221]	valid_0's multi_logloss: 0.837141
[222]	valid_0's multi_logloss: 0.836113
[223]	valid_0's multi_logloss: 0.83497
[224]	valid_0's multi_logloss: 0.83396
[225]	valid_0's multi_logloss: 0.833002
[226]	valid_0's multi_logloss: 0.831999
[227]	valid_0's multi_logloss: 0.830953
[228]	valid_0's multi_logloss: 0.829918
[229]	valid_0's multi_logloss: 0.828955
[230]	valid_0's multi_logloss: 0.827927
[231]	valid_0's multi_logloss: 0.826883
[232]	valid_0's multi_logloss: 0.825871
[233]	valid_0's multi_logloss: 0.824903
[234]	valid_0's multi_logloss: 0.823984
[23

[416]	valid_0's multi_logloss: 0.705618
[417]	valid_0's multi_logloss: 0.705175
[418]	valid_0's multi_logloss: 0.704705
[419]	valid_0's multi_logloss: 0.704342
[420]	valid_0's multi_logloss: 0.703914
[421]	valid_0's multi_logloss: 0.703507
[422]	valid_0's multi_logloss: 0.703137
[423]	valid_0's multi_logloss: 0.702743
[424]	valid_0's multi_logloss: 0.702361
[425]	valid_0's multi_logloss: 0.701961
[426]	valid_0's multi_logloss: 0.701535
[427]	valid_0's multi_logloss: 0.701114
[428]	valid_0's multi_logloss: 0.700793
[429]	valid_0's multi_logloss: 0.700445
[430]	valid_0's multi_logloss: 0.700071
[431]	valid_0's multi_logloss: 0.699699
[432]	valid_0's multi_logloss: 0.699283
[433]	valid_0's multi_logloss: 0.698904
[434]	valid_0's multi_logloss: 0.698501
[435]	valid_0's multi_logloss: 0.698096
[436]	valid_0's multi_logloss: 0.69768
[437]	valid_0's multi_logloss: 0.697247
[438]	valid_0's multi_logloss: 0.696835
[439]	valid_0's multi_logloss: 0.69645
[440]	valid_0's multi_logloss: 0.696099
[4

[622]	valid_0's multi_logloss: 0.649197
[623]	valid_0's multi_logloss: 0.649014
[624]	valid_0's multi_logloss: 0.648863
[625]	valid_0's multi_logloss: 0.648722
[626]	valid_0's multi_logloss: 0.648549
[627]	valid_0's multi_logloss: 0.648396
[628]	valid_0's multi_logloss: 0.648204
[629]	valid_0's multi_logloss: 0.648054
[630]	valid_0's multi_logloss: 0.647924
[631]	valid_0's multi_logloss: 0.647769
[632]	valid_0's multi_logloss: 0.647621
[633]	valid_0's multi_logloss: 0.64749
[634]	valid_0's multi_logloss: 0.647323
[635]	valid_0's multi_logloss: 0.647164
[636]	valid_0's multi_logloss: 0.646982
[637]	valid_0's multi_logloss: 0.646814
[638]	valid_0's multi_logloss: 0.646664
[639]	valid_0's multi_logloss: 0.64651
[640]	valid_0's multi_logloss: 0.646346
[641]	valid_0's multi_logloss: 0.646174
[642]	valid_0's multi_logloss: 0.645972
[643]	valid_0's multi_logloss: 0.645829
[644]	valid_0's multi_logloss: 0.645632
[645]	valid_0's multi_logloss: 0.645477
[646]	valid_0's multi_logloss: 0.645329
[6

[828]	valid_0's multi_logloss: 0.62565
[829]	valid_0's multi_logloss: 0.625582
[830]	valid_0's multi_logloss: 0.625522
[831]	valid_0's multi_logloss: 0.625455
[832]	valid_0's multi_logloss: 0.625428
[833]	valid_0's multi_logloss: 0.625355
[834]	valid_0's multi_logloss: 0.625279
[835]	valid_0's multi_logloss: 0.62525
[836]	valid_0's multi_logloss: 0.625198
[837]	valid_0's multi_logloss: 0.625128
[838]	valid_0's multi_logloss: 0.625071
[839]	valid_0's multi_logloss: 0.624999
[840]	valid_0's multi_logloss: 0.624933
[841]	valid_0's multi_logloss: 0.624893
[842]	valid_0's multi_logloss: 0.624839
[843]	valid_0's multi_logloss: 0.62478
[844]	valid_0's multi_logloss: 0.624758
[845]	valid_0's multi_logloss: 0.624701
[846]	valid_0's multi_logloss: 0.624661
[847]	valid_0's multi_logloss: 0.624612
[848]	valid_0's multi_logloss: 0.624537
[849]	valid_0's multi_logloss: 0.624464
[850]	valid_0's multi_logloss: 0.624392
[851]	valid_0's multi_logloss: 0.624353
[852]	valid_0's multi_logloss: 0.624275
[85

[1033]	valid_0's multi_logloss: 0.616826
[1034]	valid_0's multi_logloss: 0.616798
[1035]	valid_0's multi_logloss: 0.616789
[1036]	valid_0's multi_logloss: 0.61676
[1037]	valid_0's multi_logloss: 0.616743
[1038]	valid_0's multi_logloss: 0.616721
[1039]	valid_0's multi_logloss: 0.616704
[1040]	valid_0's multi_logloss: 0.616668
[1041]	valid_0's multi_logloss: 0.616645
[1042]	valid_0's multi_logloss: 0.616638
[1043]	valid_0's multi_logloss: 0.616612
[1044]	valid_0's multi_logloss: 0.616592
[1045]	valid_0's multi_logloss: 0.616579
[1046]	valid_0's multi_logloss: 0.616592
[1047]	valid_0's multi_logloss: 0.61656
[1048]	valid_0's multi_logloss: 0.616535
[1049]	valid_0's multi_logloss: 0.616512
[1050]	valid_0's multi_logloss: 0.616506
[1051]	valid_0's multi_logloss: 0.616478
[1052]	valid_0's multi_logloss: 0.616475
[1053]	valid_0's multi_logloss: 0.616458
[1054]	valid_0's multi_logloss: 0.61643
[1055]	valid_0's multi_logloss: 0.616406
[1056]	valid_0's multi_logloss: 0.616375
[1057]	valid_0's mu

ValueError: Number of features of the model must match the input. Model n_features_ is 2892 and input n_features is 1990 

In [34]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  34.5542%
Log Loss (CV):   1.4107


## Bag-of-Words_Tfidf 피처 생성2

In [20]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/silbu73/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/silbu73/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_tfi = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X_tfi.shape, X_tst.shape)

(54879, 5897) (19617, 5897)


In [22]:
X_tfi[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

In [23]:
y=trn.author.values
X_tfi = X_tfi.astype('float32') 
y = y.astype('float32')
X_tst = X_tst.astype('float32')

## 하이퍼 파라미터 튜닝

In [24]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
from sklearn.model_selection import StratifiedKFold, train_test_split

In [26]:
X_trn, X_val, y_trn, y_val = train_test_split(X_tfi, y, test_size=.2, random_state=42)

In [28]:
params = {
    "objective": "multiclass",
    "n_estimators": 1000,
    "subsample_freq": 1,
    "random_state": 42,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.009), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [15, 31, 63, 127, 255]),
    "colsample_bytree": hp.quniform("colsample_bytree", .4, .9, 0.1),
    "subsample": hp.quniform("subsample", .4, .9, 0.1),
}

In [29]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|██████████| 10/10 [24:27<00:00, 146.75s/trial, best loss: 0.6236845738732151]
{'objective': 'multiclass', 'n_estimators': 1000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.8, 'learning_rate': 0.04147603536728929, 'num_leaves': 127, 'subsample': 0.9}


## Stratified K-Fold Cross Validation

In [30]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Lgbm 모델 학습_tfidf

In [31]:
p = np.zeros((X_tfi.shape[0], 5))
p_tst = np.zeros((X_tst.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(X_tfi, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=1000,
                             num_leaves=127,
                             learning_rate=0.04147603536728929,
                             subsample=.9,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.8,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(X_tfi[i_trn], y[i_trn],eval_set=[(X_tfi[i_val], y[i_val])], eval_metric='multiclass')
    #early_stopping_rounds=15
    p[i_val, :] = clf.predict_proba(X_tfi[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(X_tst) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

training model for CV #1
[1]	valid_0's multi_logloss: 1.51853
[2]	valid_0's multi_logloss: 1.47259
[3]	valid_0's multi_logloss: 1.4322
[4]	valid_0's multi_logloss: 1.39536
[5]	valid_0's multi_logloss: 1.36216
[6]	valid_0's multi_logloss: 1.33023
[7]	valid_0's multi_logloss: 1.30199
[8]	valid_0's multi_logloss: 1.27433
[9]	valid_0's multi_logloss: 1.24921
[10]	valid_0's multi_logloss: 1.22659
[11]	valid_0's multi_logloss: 1.2053
[12]	valid_0's multi_logloss: 1.18497
[13]	valid_0's multi_logloss: 1.16609
[14]	valid_0's multi_logloss: 1.14904
[15]	valid_0's multi_logloss: 1.13281
[16]	valid_0's multi_logloss: 1.11681
[17]	valid_0's multi_logloss: 1.10185
[18]	valid_0's multi_logloss: 1.08852
[19]	valid_0's multi_logloss: 1.07499
[20]	valid_0's multi_logloss: 1.06167
[21]	valid_0's multi_logloss: 1.04933
[22]	valid_0's multi_logloss: 1.03755
[23]	valid_0's multi_logloss: 1.02631
[24]	valid_0's multi_logloss: 1.01568
[25]	valid_0's multi_logloss: 1.00536
[26]	valid_0's multi_logloss: 0.9959

[209]	valid_0's multi_logloss: 0.623669
[210]	valid_0's multi_logloss: 0.62338
[211]	valid_0's multi_logloss: 0.623047
[212]	valid_0's multi_logloss: 0.622788
[213]	valid_0's multi_logloss: 0.622633
[214]	valid_0's multi_logloss: 0.622399
[215]	valid_0's multi_logloss: 0.622182
[216]	valid_0's multi_logloss: 0.621928
[217]	valid_0's multi_logloss: 0.621654
[218]	valid_0's multi_logloss: 0.621347
[219]	valid_0's multi_logloss: 0.621139
[220]	valid_0's multi_logloss: 0.620782
[221]	valid_0's multi_logloss: 0.620531
[222]	valid_0's multi_logloss: 0.620305
[223]	valid_0's multi_logloss: 0.620046
[224]	valid_0's multi_logloss: 0.619708
[225]	valid_0's multi_logloss: 0.619575
[226]	valid_0's multi_logloss: 0.619332
[227]	valid_0's multi_logloss: 0.619064
[228]	valid_0's multi_logloss: 0.618829
[229]	valid_0's multi_logloss: 0.618759
[230]	valid_0's multi_logloss: 0.618612
[231]	valid_0's multi_logloss: 0.618383
[232]	valid_0's multi_logloss: 0.618281
[233]	valid_0's multi_logloss: 0.618033
[

[415]	valid_0's multi_logloss: 0.614942
[416]	valid_0's multi_logloss: 0.615044
[417]	valid_0's multi_logloss: 0.615089
[418]	valid_0's multi_logloss: 0.615122
[419]	valid_0's multi_logloss: 0.615129
[420]	valid_0's multi_logloss: 0.615308
[421]	valid_0's multi_logloss: 0.615362
[422]	valid_0's multi_logloss: 0.615496
[423]	valid_0's multi_logloss: 0.615574
[424]	valid_0's multi_logloss: 0.615587
[425]	valid_0's multi_logloss: 0.615672
[426]	valid_0's multi_logloss: 0.615628
[427]	valid_0's multi_logloss: 0.615748
[428]	valid_0's multi_logloss: 0.615922
[429]	valid_0's multi_logloss: 0.616028
[430]	valid_0's multi_logloss: 0.616151
[431]	valid_0's multi_logloss: 0.616216
[432]	valid_0's multi_logloss: 0.616428
[433]	valid_0's multi_logloss: 0.616569
[434]	valid_0's multi_logloss: 0.616652
[435]	valid_0's multi_logloss: 0.616783
[436]	valid_0's multi_logloss: 0.616856
[437]	valid_0's multi_logloss: 0.616929
[438]	valid_0's multi_logloss: 0.617158
[439]	valid_0's multi_logloss: 0.617274


[621]	valid_0's multi_logloss: 0.640706
[622]	valid_0's multi_logloss: 0.640864
[623]	valid_0's multi_logloss: 0.641038
[624]	valid_0's multi_logloss: 0.641355
[625]	valid_0's multi_logloss: 0.641437
[626]	valid_0's multi_logloss: 0.641713
[627]	valid_0's multi_logloss: 0.641915
[628]	valid_0's multi_logloss: 0.642125
[629]	valid_0's multi_logloss: 0.642359
[630]	valid_0's multi_logloss: 0.64251
[631]	valid_0's multi_logloss: 0.642725
[632]	valid_0's multi_logloss: 0.64275
[633]	valid_0's multi_logloss: 0.642955
[634]	valid_0's multi_logloss: 0.643086
[635]	valid_0's multi_logloss: 0.643278
[636]	valid_0's multi_logloss: 0.643481
[637]	valid_0's multi_logloss: 0.643718
[638]	valid_0's multi_logloss: 0.643889
[639]	valid_0's multi_logloss: 0.643995
[640]	valid_0's multi_logloss: 0.644154
[641]	valid_0's multi_logloss: 0.644439
[642]	valid_0's multi_logloss: 0.644625
[643]	valid_0's multi_logloss: 0.64477
[644]	valid_0's multi_logloss: 0.64506
[645]	valid_0's multi_logloss: 0.64525
[646]

[827]	valid_0's multi_logloss: 0.678958
[828]	valid_0's multi_logloss: 0.679148
[829]	valid_0's multi_logloss: 0.679282
[830]	valid_0's multi_logloss: 0.679431
[831]	valid_0's multi_logloss: 0.679667
[832]	valid_0's multi_logloss: 0.680007
[833]	valid_0's multi_logloss: 0.680207
[834]	valid_0's multi_logloss: 0.680383
[835]	valid_0's multi_logloss: 0.680667
[836]	valid_0's multi_logloss: 0.680882
[837]	valid_0's multi_logloss: 0.681196
[838]	valid_0's multi_logloss: 0.681421
[839]	valid_0's multi_logloss: 0.681634
[840]	valid_0's multi_logloss: 0.681761
[841]	valid_0's multi_logloss: 0.682007
[842]	valid_0's multi_logloss: 0.682208
[843]	valid_0's multi_logloss: 0.68244
[844]	valid_0's multi_logloss: 0.682632
[845]	valid_0's multi_logloss: 0.682837
[846]	valid_0's multi_logloss: 0.682997
[847]	valid_0's multi_logloss: 0.683297
[848]	valid_0's multi_logloss: 0.683527
[849]	valid_0's multi_logloss: 0.683723
[850]	valid_0's multi_logloss: 0.683942
[851]	valid_0's multi_logloss: 0.684128
[

...

[1176]	valid_0's multi_logloss: 0.760666
[1177]	valid_0's multi_logloss: 0.760944
[1178]	valid_0's multi_logloss: 0.761196
[1179]	valid_0's multi_logloss: 0.761332
[1180]	valid_0's multi_logloss: 0.761597
[1181]	valid_0's multi_logloss: 0.761813
[1182]	valid_0's multi_logloss: 0.762093
[1183]	valid_0's multi_logloss: 0.762381
[1184]	valid_0's multi_logloss: 0.762516
[1185]	valid_0's multi_logloss: 0.762685
[1186]	valid_0's multi_logloss: 0.76293
[1187]	valid_0's multi_logloss: 0.763176
[1188]	valid_0's multi_logloss: 0.763336
[1189]	valid_0's multi_logloss: 0.763612
[1190]	valid_0's multi_logloss: 0.763912
[1191]	valid_0's multi_logloss: 0.764059
[1192]	valid_0's multi_logloss: 0.764356
[1193]	valid_0's multi_logloss: 0.764494
[1194]	valid_0's multi_logloss: 0.764811
[1195]	valid_0's multi_logloss: 0.765044
[1196]	valid_0's multi_logloss: 0.765239
[1197]	valid_0's multi_logloss: 0.765372
[1198]	valid_0's multi_logloss: 0.765596
[1199]	valid_0's multi_logloss: 0.765869
[1200]	valid_0's 

[1377]	valid_0's multi_logloss: 0.80677
[1378]	valid_0's multi_logloss: 0.806956
[1379]	valid_0's multi_logloss: 0.807124
[1380]	valid_0's multi_logloss: 0.807336
[1381]	valid_0's multi_logloss: 0.807548
[1382]	valid_0's multi_logloss: 0.807849
[1383]	valid_0's multi_logloss: 0.808077
[1384]	valid_0's multi_logloss: 0.808291
[1385]	valid_0's multi_logloss: 0.808631
[1386]	valid_0's multi_logloss: 0.80884
[1387]	valid_0's multi_logloss: 0.809104
[1388]	valid_0's multi_logloss: 0.809422
[1389]	valid_0's multi_logloss: 0.809634
[1390]	valid_0's multi_logloss: 0.809891
[1391]	valid_0's multi_logloss: 0.810272
[1392]	valid_0's multi_logloss: 0.810475
[1393]	valid_0's multi_logloss: 0.810673
[1394]	valid_0's multi_logloss: 0.810826
[1395]	valid_0's multi_logloss: 0.811115
[1396]	valid_0's multi_logloss: 0.811417
[1397]	valid_0's multi_logloss: 0.811685
[1398]	valid_0's multi_logloss: 0.81188
[1399]	valid_0's multi_logloss: 0.812133
[1400]	valid_0's multi_logloss: 0.812423
[1401]	valid_0's mu

[1578]	valid_0's multi_logloss: 0.855907
[1579]	valid_0's multi_logloss: 0.856185
[1580]	valid_0's multi_logloss: 0.856336
[1581]	valid_0's multi_logloss: 0.856585
[1582]	valid_0's multi_logloss: 0.85676
[1583]	valid_0's multi_logloss: 0.857046
[1584]	valid_0's multi_logloss: 0.857259
[1585]	valid_0's multi_logloss: 0.857556
[1586]	valid_0's multi_logloss: 0.857779
[1587]	valid_0's multi_logloss: 0.85804
[1588]	valid_0's multi_logloss: 0.858235
[1589]	valid_0's multi_logloss: 0.858437
[1590]	valid_0's multi_logloss: 0.858611
[1591]	valid_0's multi_logloss: 0.858861
[1592]	valid_0's multi_logloss: 0.859157
[1593]	valid_0's multi_logloss: 0.859449
[1594]	valid_0's multi_logloss: 0.859694
[1595]	valid_0's multi_logloss: 0.859931
[1596]	valid_0's multi_logloss: 0.860178
[1597]	valid_0's multi_logloss: 0.860511
[1598]	valid_0's multi_logloss: 0.860761
[1599]	valid_0's multi_logloss: 0.860958
[1600]	valid_0's multi_logloss: 0.861171
[1601]	valid_0's multi_logloss: 0.861397
[1602]	valid_0's m

[1779]	valid_0's multi_logloss: 0.905696
[1780]	valid_0's multi_logloss: 0.905936
[1781]	valid_0's multi_logloss: 0.90613
[1782]	valid_0's multi_logloss: 0.906402
[1783]	valid_0's multi_logloss: 0.906656
[1784]	valid_0's multi_logloss: 0.906926
[1785]	valid_0's multi_logloss: 0.907142
[1786]	valid_0's multi_logloss: 0.907233
[1787]	valid_0's multi_logloss: 0.907458
[1788]	valid_0's multi_logloss: 0.907728
[1789]	valid_0's multi_logloss: 0.907912
[1790]	valid_0's multi_logloss: 0.908172
[1791]	valid_0's multi_logloss: 0.908375
[1792]	valid_0's multi_logloss: 0.908727
[1793]	valid_0's multi_logloss: 0.909024
[1794]	valid_0's multi_logloss: 0.909254
[1795]	valid_0's multi_logloss: 0.909517
[1796]	valid_0's multi_logloss: 0.909796
[1797]	valid_0's multi_logloss: 0.910113
[1798]	valid_0's multi_logloss: 0.910305
[1799]	valid_0's multi_logloss: 0.910522
[1800]	valid_0's multi_logloss: 0.910752
[1801]	valid_0's multi_logloss: 0.910951
[1802]	valid_0's multi_logloss: 0.911161
[1803]	valid_0's 

[1980]	valid_0's multi_logloss: 0.95455
[1981]	valid_0's multi_logloss: 0.954803
[1982]	valid_0's multi_logloss: 0.95499
[1983]	valid_0's multi_logloss: 0.955224
[1984]	valid_0's multi_logloss: 0.955488
[1985]	valid_0's multi_logloss: 0.955834
[1986]	valid_0's multi_logloss: 0.956152
[1987]	valid_0's multi_logloss: 0.95628
[1988]	valid_0's multi_logloss: 0.956496
[1989]	valid_0's multi_logloss: 0.956741
[1990]	valid_0's multi_logloss: 0.956915
[1991]	valid_0's multi_logloss: 0.957213
[1992]	valid_0's multi_logloss: 0.957486
[1993]	valid_0's multi_logloss: 0.957773
[1994]	valid_0's multi_logloss: 0.95801
[1995]	valid_0's multi_logloss: 0.958228
[1996]	valid_0's multi_logloss: 0.95858
[1997]	valid_0's multi_logloss: 0.958785
[1998]	valid_0's multi_logloss: 0.95903
[1999]	valid_0's multi_logloss: 0.959222
[2000]	valid_0's multi_logloss: 0.959543


In [32]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  76.0947%
Log Loss (CV):   0.9759


#Accuracy (CV):  76.9274%  
#Log Loss (CV):   0.6212  ...contest : 0.4099    
lgb.LGBMClassifier(objective='multiclass', n_estimators=3000,   num_leaves=256,   learning_rate=0.009,  subsample=.8,  subsample_freq=1, num_iterations=2000,
colsample_bytree=.6,   random_state=42,       n_jobs=-1)  
                             
#Accuracy (CV):  76.0947%  
#Log Loss (CV):   0.9759...contest :  0.4318  
lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=1000,
                             num_leaves=127,
                             learning_rate=0.04147603536728929,
                             subsample=.9,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.8,
                             random_state=42,
                             n_jobs=-1)
                             
                           

## 파일제출

In [33]:
# submission
sample_submission[['0','1','2','3','4']] = p_tst
sample_submission

,index,0,1,2,3,4
0,0,0.000509,0.980555,1.852706e-02,4.064327e-04,2.637948e-06
1,1,0.000430,0.998412,5.333983e-05,8.086088e-05,1.023672e-03
2,2,0.999296,0.000016,1.568217e-04,8.760408e-05,4.438005e-04
3,3,0.000003,0.000015,9.999747e-01,5.950140e-06,1.967738e-06
4,4,0.723953,0.057970,1.467240e-01,6.335313e-02,8.000149e-03
...,...,...,...,...,...,...
19612,19612,0.000029,0.999970,7.696671e-10,2.435033e-07,9.413262e-08
19613,19613,0.030582,0.000007,2.349613e-04,1.622589e-05,9.691605e-01
19614,19614,0.000007,0.999992,4.403896e-10,1.034265e-06,3.914624e-09
19615,19615,0.000159,0.999549,5.806682e-05,2.336879e-04,8.621616e-07


In [34]:
sample_submission.to_csv('thisissubmission.csv', index = False, encoding = 'utf-8')

## test 데이터 셋 이용하기
(대회 규정상 이를 제출하면 안됨!!)

In [15]:
df = pd.concat([train, test], axis=0)

In [16]:
print(df.shape)
df.tail()

(74496, 3)


,index,text,author
19612,19612,"At the end of another day or two, odin growing...",NaN
19613,19613,"All afternoon we sat together, mostly in silen...",NaN
19614,19614,"odin, having carried his thanks to odin, proc...",NaN
19615,19615,"Soon after this, upon odin's leaving the room,...",NaN
19616,19616,"And all the worse for the doomed man, that the...",NaN


In [17]:
df.fillna(-1, inplace=True)
df.tail()

,index,text,author
19612,19612,"At the end of another day or two, odin growing...",-1.0
19613,19613,"All afternoon we sat together, mostly in silen...",-1.0
19614,19614,"odin, having carried his thanks to odin, proc...",-1.0
19615,19615,"Soon after this, upon odin's leaving the room,...",-1.0
19616,19616,"And all the worse for the doomed man, that the...",-1.0


In [18]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

df['text']=df['text'].apply(alpha_num)

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
df=df['text'].values

In [21]:
for i, document in enumerate(df):
    words = word_tokenize(document)
    #print(words) #
    clean_words = []
    for word in words:
        word = word.lower()                         #대소분자 변환
        if word not in stopwords.words('english'): #불용어 제거
            stemmer = SnowballStemmer('english')
            word = stemmer.stem(word)              #어간 추출
            clean_words.append(word)
    document = ' '.join(clean_words)
    df[i] = document

KeyboardInterrupt: 

In [ ]:
df = CountVectorizer(tokenizer=word_tokenize, stop_words='english', min_df=50).fit_transform(df)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2, # 토큰이 나타날 최소 문서 개수
                             ngram_range=(4, 4),
                             max_features = 20000
                            )
vectorizer
pipeline = Pipeline([
    ('vect', vectorizer),
])


%time train_data_features = pipeline.fit_transform(df_train['review_clean'])
%time test_data_features = pipeline.fit_transform(df_test['review_clean'])

In [ ]:
# train test 분리
y_train = df['author'].values

In [ ]:
x_train = df[:54879]                #train을 위한 독립 320000개     
x_test = df[54879:]              #test을 위한 독립 80000개

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train=y_train[:54879]

In [ ]:
x_train = x_train.astype('float32') 
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')   

In [ ]:
p_val = np.zeros((x_train.shape[0], 5))
p_tst = np.zeros((x_test.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(x_train, y_train), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=3000,
                             num_leaves=256,
                             learning_rate=0.009,
                             subsample=.8,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.6,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(x_train[i_trn], y_train[i_trn],eval_set=[(x_train[i_val], y_train[i_val])], eval_metric='multiclass',early_stopping_rounds=15)
    
    p_val[i_val, :] = clf.predict_proba(x_train[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(x_test) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

In [ ]:
p_tst

In [ ]:
print(f'Log loss (CV) : {log_loss(pd.get_dummies(y_train),p_tst):8.4f}')

In [ ]:
print(f'Log loss (CV) : {log_loss(pd.get_dummies(y_train),p_val):8.4f}')